In [5]:
import numpy as np
import pandas as pd
import networkx as nx

df = pd.read_csv("C:/Users/surup/Downloads/data_and_path/myDB.csv")

def find_optimal_path_with_coordinates(flight_source, flight_destination, excluded_airport, included_airport):

    # Average speed and stopover time
    avg_speed = 900  # Average speed in km/hr
    stopover_time = 2  # Stopover time in hours

    # Check if source or destination is the excluded_airport
    if flight_source == excluded_airport:
        return "Flight cannot start from " + excluded_airport
    elif flight_destination == excluded_airport:
        return "There are no flights to " + excluded_airport
    else:
        # Create graph from DataFrame
        G = nx.from_pandas_edgelist(df, 's_airport', 'd_airport', ['airline', 'airline_id', 'distance(Km)'])

        # Function to find all routes from source to destination including indirect flights
        def find_all_routes(graph, source, destination):
            try:
                return list(nx.all_simple_paths(graph, source, destination))
            except nx.NetworkXNoPath:
                return []

        # Function to calculate total distance of a route
        def calculate_route_distance(route):
            distance = 0
            for i in range(len(route) - 1):
                distance += G[route[i]][route[i+1]]['distance(Km)']
            return distance

        # Function to calculate total time of a route
        def calculate_route_time(route):
            total_distance = calculate_route_distance(route)
            total_time = total_distance / avg_speed  # Time in hours
            total_time += (len(route) - 2) * stopover_time  # Add stopover time
            return total_time

        # Function to get latitude and longitude of airports in a route
        def get_airport_coordinates(route):
            airport_coordinates = []
            for airport_code in route:
                airport_info = df[(df['s_airport'] == airport_code) | (df['d_airport'] == airport_code)].iloc[0]
                airport_coordinates.append((airport_info['s_latitude'], airport_info['s_longitude']))
            return airport_coordinates

        # Find all routes from source to destination
        all_routes = find_all_routes(G, flight_source, flight_destination)

        # Filter routes based on excluded_airport
        if excluded_airport:
            all_routes = [route for route in all_routes if excluded_airport not in route]

        # Further filter routes based on included_airport
        if included_airport:
            all_routes = [route for route in all_routes if included_airport in route]

        # Sort routes by total time
        sorted_routes = sorted(all_routes, key=calculate_route_time)

        # Prepare results with coordinates
        results_with_coordinates = []
        for i, route in enumerate(sorted_routes, start=1):
            total_distance = calculate_route_distance(route)
            total_time = calculate_route_time(route)
            total_stopovers = len(route) - 2
            coordinates = get_airport_coordinates(route)
            results_with_coordinates.append({
                'Route': route,
                'Total Distance (km)': total_distance,
                'Total Stopovers': total_stopovers,
                'Total Time (hours)': round(total_time, 2),
                'Coordinates': coordinates
            })

        # Determine the optimal path with coordinates
        if sorted_routes:
            optimal_path_with_coordinates = results_with_coordinates[0]
        else:
            optimal_path_with_coordinates = "No routes found that satisfy the criteria."

        return results_with_coordinates, optimal_path_with_coordinates

# Get user input
flight_source = input("Enter flight source: ")
flight_destination = input("Enter flight destination: ")
excluded_airport = input("Enter excluded airport (leave empty if none): ")
included_airport = input("Enter included airport (leave empty if none): ")

# Call the function
routes_with_coordinates, optimal_path_with_coordinates = find_optimal_path_with_coordinates(flight_source, flight_destination, excluded_airport, included_airport)

# Print the results
print("\nAll routes from", flight_source, "to", flight_destination, "considering the constraints:")
for route_info in routes_with_coordinates:
    print(route_info)

print("\nOptimal Path Details:")
print(optimal_path_with_coordinates)



All routes from AER to LED considering the constraints:
{'Route': ['AER', 'KZN', 'SVX', 'NBC', 'LED'], 'Total Distance (km)': 4157.938582004321, 'Total Stopovers': 3, 'Total Time (hours)': 10.62, 'Coordinates': [(43.449902, 39.9566), (43.449902, 39.9566), (56.743099212646, 60.802700042725), (55.564701080322266, 52.09249877929688), (59.80030059814453, 30.262500762939453)]}
{'Route': ['AER', 'KZN', 'SVX', 'NJC', 'UUA', 'LED'], 'Total Distance (km)': 6264.180278645278, 'Total Stopovers': 4, 'Total Time (hours)': 14.96, 'Coordinates': [(43.449902, 39.9566), (43.449902, 39.9566), (56.743099212646, 60.802700042725), (60.94929885864258, 76.48359680175781), (54.63999938964844, 52.801700592041016), (59.80030059814453, 30.262500762939453)]}
{'Route': ['AER', 'KZN', 'SVX', 'NBC', 'DME', 'UUA', 'LED'], 'Total Distance (km)': 6092.924088751256, 'Total Stopovers': 5, 'Total Time (hours)': 16.77, 'Coordinates': [(43.449902, 39.9566), (43.449902, 39.9566), (56.743099212646, 60.802700042725), (55.5647